In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sbs
import datetime
import pymc as pm
import statsmodels.api as sm
from scipy.stats.stats import pearsonr
import matplotlib as mpl
pd.options.display.max_columns = 50

In [4]:
import pickle
def save_obj(obj, name):
    with open('C:/Users/pc/Desktop/SG_project/Pickled/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name):
    with open('C:/Users/pc/Desktop/SG_project/Pickled/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [20]:
dictionary_commodities = load_obj("dictionary_commodities")
dictionary_financials = load_obj("dictionary_financials")

In [21]:
dictionary_commodities.keys()

dict_keys(['CORN - CHICAGO BOARD OF TRADE', 'OATS - CHICAGO BOARD OF TRADE', 'COTTON NO. 2 - ICE FUTURES U.S.', 'ROUGH RICE - CHICAGO BOARD OF TRADE', 'LEAN HOGS - CHICAGO MERCANTILE EXCHANGE', 'LIVE CATTLE - CHICAGO MERCANTILE EXCHANGE', 'FEEDER CATTLE - CHICAGO MERCANTILE EXCHANGE', 'SOYBEANS - CHICAGO BOARD OF TRADE', 'SOYBEAN OIL - CHICAGO BOARD OF TRADE', 'SOYBEAN MEAL - CHICAGO BOARD OF TRADE', 'COCOA - ICE FUTURES U.S.', 'SUGAR NO. 11 - ICE FUTURES U.S.', 'COFFEE C - ICE FUTURES U.S.', 'CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE', 'GOLD - COMMODITY EXCHANGE INC.', 'NATURAL GAS - NEW YORK MERCANTILE EXCHANGE', 'NATURAL GAS ICE HENRY HUB - ICE FUTURES ENERGY DIV', 'COPPER-GRADE #1 - COMMODITY EXCHANGE INC.', 'FRZN CONCENTRATED ORANGE JUICE - ICE FUTURES U.S.'])

In [22]:
dictionary_financials.keys()

dict_keys(['E-MINI S&P 500 STOCK INDEX - CHICAGO MERCANTILE EXCHANGE', 'VIX FUTURES - CBOE FUTURES EXCHANGE', 'EURO FX - CHICAGO MERCANTILE EXCHANGE', '3-MONTH EURODOLLARS - CHICAGO MERCANTILE EXCHANGE', 'BRITISH POUND STERLING - CHICAGO MERCANTILE EXCHANGE', 'JAPANESE YEN - CHICAGO MERCANTILE EXCHANGE', '2-YEAR U.S. TREASURY NOTES - CHICAGO BOARD OF TRADE', '10-YEAR U.S. TREASURY NOTES - CHICAGO BOARD OF TRADE', '5-YEAR U.S. TREASURY NOTES - CHICAGO BOARD OF TRADE', 'CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE'])

In [7]:
df = load_obj("df_generic1_prices")
df.head()

,snp_futeres_price,vix_futeres_price,yen_futeres_price,two_years_us_yield,five_years_us_yield,ten_years_us_yield,euro_dollar_futeres_price,wheat_hrw_futeres_price,live_cattle_futeres_price,feeder_cattle_futeres_price,gold_futeres_price,coffee_futeres_price,nut_gas_ice_futeres_price,nut_gas_nymex_futeres_price,crude_oil_ls_futeres_price,soybean_oil_futeres_price,corn_futeres_price,frzn_orange_juice_futures_price,sugar_futures_price,copper_futures_price,two_tnote_futeres_price,five_tnote_futeres_price,ten_tnote_futeres_price,SG_CTA
2010-10-01,1142.25,24.75,120.07,0.4145,1.2565,2.5097,95.845,689.25,95.950,111.800,1316.1,181.10,46.76,3.797,81.58,43.49,465.75,155.65,23.36,369.05,109.757812,120.906250,126.078125,2010.976434
2010-10-04,1134.75,25.15,120.03,0.4066,1.2241,2.4758,95.890,683.00,95.100,110.500,1315.4,172.50,46.38,3.727,81.47,43.10,471.50,154.65,22.99,366.40,109.773438,121.093750,126.468750,2005.892686
2010-10-05,1154.75,23.65,120.28,0.3987,1.1917,2.4722,95.875,701.00,95.550,109.550,1338.9,177.00,47.12,3.743,82.82,43.47,491.00,158.50,23.74,372.65,109.773438,121.187500,126.640625,2030.601272
2010-10-06,1155.75,23.25,120.65,0.3829,1.1593,2.3976,95.925,697.50,94.850,109.200,1346.4,175.55,46.77,3.865,83.23,43.44,488.50,150.45,23.54,375.30,109.828125,121.359375,127.093750,2041.925935
2010-10-07,1156.50,22.95,121.52,0.3512,1.1270,2.3833,95.925,699.50,95.125,108.925,1333.9,173.45,46.54,3.617,81.67,43.79,498.25,151.20,25.16,367.95,109.882812,121.484375,127.140625,2042.460920


In [13]:
def get_change_bool(df):
    # create difference column
    df = pd.DataFrame(df)
    df["change"] = df - df.shift(1)
    # drop first row because nan is now present in the "change" column 
    df.drop(df.index[0], inplace=True)
    df["change_bool"] = 0
    # mark days with positive or no change as 0
    # mark days with negative change as 1
    
    df.loc[df["change"] < 0, "change_bool"] = 1
    # crop the data to the same time period as cftc data
    start = load_obj("cftc_index")[0]; end = load_obj("cftc_index")[-1]
    df = df.loc[start:end]
    return df

In [9]:
def get_neg_ret(df):
    df = get_change_bool(df)
    # parameter "n" is the number of consecutive days with negative return
    # column consec_day represents for how many consecutive days there was a decline 
    # column neg_return represents negative cumultive sum 
    if not isinstance(df, pd.DataFrame):
        df = pd.DataFrame(df)
    
    df["cumsum_bool"] = df['change_bool'].cumsum() 
    df["consec_days"] = df['cumsum_bool'].sub(df['cumsum_bool'].mask(df['change_bool'] != 0).ffill(), 
                                        fill_value=0).astype(int)  
    
    df["cumsum_bool_ret"] = df['change'].cumsum() 
    df["neg_return_cum"] = df['cumsum_bool_ret'].sub(df['cumsum_bool_ret'].mask(df['change_bool'] != 0).ffill(), 
                                        fill_value=0) 

    df.drop(["cumsum_bool_ret", 'cumsum_bool', 'change_bool'], 1, inplace=True)

    df["pct_neg_ret_cum"] = round(df["neg_return_cum"]/df[df.columns[0]] * 100, 4)
    df["pct_ret"] = round(df["change"]/df[df.columns[0]] * 100, 4)
    
    return df

In [28]:
def get_neg_df(df, n):
    from pandas.tseries.offsets import BDay
    df = get_neg_ret(df)
    df_return = pd.DataFrame()
    #ranges_list = []
    for index in df[df['consec_days'] == n].index:
        # index is the last day in the sequence 
        start = index - BDay(n-1)
        df_return = pd.concat([df_return, df.loc[start:index]])
        #ranges_list.append([start, index])
    return df_return#, ranges_list 

In [11]:
def map_neg_ret(df_positions, neg_df, df):
    #   
    #   performing a lot of arithmetic to dates, going to check for holidays, othersiwe will not be able to index df
    #   will try to index range within 2 days before and after holiday
    #
    #
    #   class USTradingCalendar is taken from 
    #   https://stackoverflow.com/questions/33094297/create-trading-holiday-calendar-with-pandas      // Pierre Boutquin
    #
    ##############################################################################
    from pandas.tseries.offsets import BDay
    import datetime as dt
    from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
        USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
        USLaborDay, USThanksgivingDay

    class USTradingCalendar(AbstractHolidayCalendar):
        rules = [
            Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
            USMartinLutherKingJr,
            USPresidentsDay,
            GoodFriday,
            USMemorialDay,
            Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
            USLaborDay,
            USThanksgivingDay,
            Holiday('Christmas', month=12, day=25, observance=nearest_workday)
        ]

    def get_trading_close_holidays(year):
        inst = USTradingCalendar()

        return inst.holidays(dt.datetime(year-1, 12, 31), dt.datetime(year, 12, 31))

    if __name__ == '__main__':
        pass    
    ##############################################################################
    
    df_return = neg_df
    df_return["position_change"] = 0
    # 0: Monday, 6: Sunday 
    mondays = neg_df.index[neg_df.index.dayofweek == 0]
    tuesdays = neg_df.index[neg_df.index.dayofweek == 1]
    fridays = neg_df.index[neg_df.index.dayofweek == 4]
    
    
    for tuesday in tuesdays:
        year = int(tuesday.year)
        if tuesday in get_trading_close_holidays(year):
            try:
                tuesday = tuesday - BDay(1)
                df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
            except:
                try:
                    tuesday = tuesday + BDay(1)
                    df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
                except:
                    try:
                        tuesday = tuesday - BDay(2)
                        df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
                    except:
                        tuesday = tuesday + BDay(2)
                        df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
        else:
            df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
    
    for monday in mondays:
        tuesday = monday + BDay(1)
        year = int(tuesday.year)
        if tuesday in get_trading_close_holidays(year):
            # find a day which will work 
            try:
                tuesday = tuesday - BDay(1)
                df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
            except:
                try:
                    tuesday = tuesday + BDay(1)
                    df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
                except:
                    try:
                        tuesday = tuesday - BDay(2)
                        df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
                    except:
                        tuesday = tuesday + BDay(2)
                        df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
            # try/except should have found an appropriate day, and assigned a value to it           
            df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
            df_return.loc[tuesday, df_return.columns[0]] = df.loc[tuesday]
            df_return.loc[tuesday, "change"] = df_return.loc[tuesday, df_return.columns[0]] - \
                    df.loc[tuesday - BDay(1)]
            df_return.loc[tuesday, "pct_ret"] = df_return.loc[tuesday, "change"]/\
                    df_return.loc[tuesday, df_return.columns[0]]
            
        else:
            df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
            df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
            df_return.loc[tuesday, df_return.columns[0]] = df.loc[tuesday]
            df_return.loc[tuesday, "change"] = df_return.loc[tuesday, df_return.columns[0]] - \
                    df.loc[tuesday - BDay(1)]
            df_return.loc[tuesday, "pct_ret"] = df_return.loc[tuesday, "change"]/\
                    df_return.loc[tuesday, df_return.columns[0]]

    
    for friday in fridays:
        tuesday = friday + BDay(2)
        year = int(tuesday.year)
        if tuesday in get_trading_close_holidays(year):
            # find a day which will work 
            try:
                tuesday = tuesday - BDay(1)
                df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
            except:
                try:
                    tuesday = tuesday + BDay(1)
                    df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
                except:
                    try:
                        tuesday = tuesday - BDay(2)
                        df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
                    except:
                        tuesday = tuesday + BDay(2)
                        df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
            # try/except should have found an appropriate day, and assigned a value to it           
            df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
            df_return.loc[tuesday, df_return.columns[0]] = df.loc[tuesday]
            df_return.loc[tuesday, "change"] = df_return.loc[tuesday, df_return.columns[0]] - \
                    df.loc[tuesday - BDay(1)]
            df_return.loc[tuesday, "pct_ret"] = df_return.loc[tuesday, "change"]/\
                    df_return.loc[tuesday, df_return.columns[0]]

                        
        else:
            df_return.loc[tuesday, "position_change"] = df_positions.loc[tuesday]
            df_return.loc[tuesday, df_return.columns[0]] = df.loc[tuesday]
            df_return.loc[tuesday, "change"] = df_return.loc[tuesday, df_return.columns[0]] - \
                    df.loc[tuesday - BDay(1)]
            df_return.loc[tuesday, "pct_ret"] = df_return.loc[tuesday, "change"]/\
                    df_return.loc[tuesday, df_return.columns[0]]*100

    
    # sort all the added indecies   
    df_return.sort_index(inplace=True)
    return df_return

In [42]:
q= get_neg_df(df["snp_futeres_price"], 8)
q

,snp_futeres_price,change,consec_days,neg_return_cum,pct_neg_ret_cum,pct_ret
2011-07-22,1341.00,-1.50,1,-1.50,-0.1119,-0.1119
2011-07-25,1333.50,-7.50,2,-9.00,-0.6749,-0.5624
2011-07-26,1326.25,-7.25,3,-16.25,-1.2253,-0.5467
2011-07-27,1299.00,-27.25,4,-43.50,-3.3487,-2.0978
2011-07-28,1296.75,-2.25,5,-45.75,-3.5281,-0.1735
2011-07-29,1288.50,-8.25,6,-54.00,-4.1909,-0.6403
2011-08-01,1279.75,-8.75,7,-62.75,-4.9033,-0.6837
2011-08-02,1247.25,-32.50,8,-95.25,-7.6368,-2.6057
2011-11-16,1231.00,-23.00,1,-23.00,-1.8684,-1.8684
2011-11-17,1214.75,-16.25,2,-39.25,-3.2311,-1.3377


In [43]:
map_neg_ret(dictionary_financials["E-MINI S&P 500 STOCK INDEX - CHICAGO MERCANTILE EXCHANGE"]\
['Change_in_Net_Positioning_Lev_Money'], q, df["snp_futeres_price"])

,snp_futeres_price,change,consec_days,neg_return_cum,pct_neg_ret_cum,pct_ret,position_change
2011-07-22,1341.00,-1.50,1.0,-1.50,-0.1119,-0.111900,0.0
2011-07-25,1333.50,-7.50,2.0,-9.00,-0.6749,-0.562400,0.0
2011-07-26,1326.25,-7.25,3.0,-16.25,-1.2253,-0.546654,-48167.0
2011-07-27,1299.00,-27.25,4.0,-43.50,-3.3487,-2.097800,0.0
2011-07-28,1296.75,-2.25,5.0,-45.75,-3.5281,-0.173500,0.0
2011-07-29,1288.50,-8.25,6.0,-54.00,-4.1909,-0.640300,0.0
2011-08-01,1279.75,-8.75,7.0,-62.75,-4.9033,-0.683700,0.0
2011-08-02,1247.25,-32.50,8.0,-95.25,-7.6368,-2.605733,131348.0
2011-11-16,1231.00,-23.00,1.0,-23.00,-1.8684,-1.868400,0.0
2011-11-17,1214.75,-16.25,2.0,-39.25,-3.2311,-1.337700,0.0


In [30]:
def get_stats(df_positions, df, min_days, max_days):
    df_return = pd.DataFrame()
    for n in range(min_days, max_days+1):
        neg_df = get_neg_df(df, n)
        mapped_df = map_neg_ret(df_positions, neg_df, df)
        df_return = pd.concat([df_return, mapped_df])
    return df_return    

In [32]:
s = get_stats(dictionary_financials["VIX FUTURES - CBOE FUTURES EXCHANGE"]\
['Change_in_Net_Positioning_Lev_Money'], df["vix_futeres_price"], 8, 10)

In [79]:
s

,vix_futeres_price,change,consec_days,neg_return_cum,pct_neg_ret_cum,pct_ret,position_change
2013-06-25,19.450,-0.700,1.0,-0.700,-3.5990,-3.599000,14509.0
2013-06-26,18.850,-0.600,2.0,-1.300,-6.8966,-3.183000,0.0
2013-06-27,18.200,-0.650,3.0,-1.950,-10.7143,-3.571400,0.0
2013-06-28,18.050,-0.150,4.0,-2.100,-11.6343,-0.831000,0.0
2013-07-01,17.650,-0.400,5.0,-2.500,-14.1643,-2.266300,0.0
2013-07-02,17.600,-0.050,6.0,-2.550,-14.4886,-0.284091,-14258.0
2013-07-03,17.250,-0.350,7.0,-2.900,-16.8116,-2.029000,0.0
2013-07-04,16.600,-0.650,8.0,-3.550,-21.3855,-3.915700,0.0
2016-07-06,16.175,-0.650,1.0,-0.650,-4.0185,-4.018500,0.0
2016-07-07,15.925,-0.250,2.0,-0.900,-5.6515,-1.569900,0.0


In [80]:
s[s['position_change'] != 0]

,vix_futeres_price,change,consec_days,neg_return_cum,pct_neg_ret_cum,pct_ret,position_change
2013-06-25,19.450,-0.70,1.0,-0.700,-3.5990,-3.599000,14509.0
2013-07-02,17.600,-0.05,6.0,-2.550,-14.4886,-0.284091,-14258.0
2016-07-12,14.275,-0.35,5.0,-2.550,-17.8634,-2.451839,4300.0
2016-07-19,12.225,-0.85,NaN,NaN,NaN,-6.952965,-18557.0
2017-07-11,12.125,-0.10,3.0,-0.900,-7.4227,-0.824742,12308.0
2017-07-18,10.125,-0.15,8.0,-2.900,-28.6420,-1.481481,-17958.0
2017-11-21,11.625,-0.45,4.0,-1.425,-12.2581,-3.870968,33879.0
2017-11-28,11.275,-0.05,NaN,NaN,NaN,-0.443459,831.0
2013-06-25,19.450,-0.70,1.0,-0.700,-3.5990,-3.599000,14509.0
2013-07-02,17.600,-0.05,6.0,-2.550,-14.4886,-0.284091,-14258.0


In [53]:
s[s['position_change'] != 0]['pct_ret']

2013-06-25   -3.599000
2013-07-02   -0.284091
2016-07-12   -2.451839
2016-07-19   -6.952965
2017-07-11   -0.824742
2017-07-18   -1.481481
2017-11-21   -3.870968
2017-11-28   -0.443459
2013-06-25   -3.599000
2013-07-02   -0.284091
2013-07-09   -1.655629
2016-07-12   -2.451839
2016-07-19   -6.952965
2017-07-11   -0.824742
2017-07-18   -1.481481
2017-11-21   -3.870968
2017-11-28   -0.443459
2013-06-25   -3.599000
2013-07-02   -0.284091
2013-07-09   -1.655629
2016-07-12   -2.451839
2016-07-19   -6.952965
Name: pct_ret, dtype: float64